In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

# Загрузка данных
df = pd.read_csv('jamb_exam_results.csv')

# Преобразование названий колонок к нижнему регистру
df.columns = df.columns.str.lower().str.replace(' ', '_')

print("Размер датасета:", df.shape)
print("\nПервые 5 строк:")
print(df.head())

# Подготовка данных
# Удаляем столбец student_id
df = df.drop('student_id', axis=1)

# Заполняем пропущенные значения нулями
df = df.fillna(0)

# Разделяем на признаки и целевую переменную
X = df.drop('jamb_score', axis=1)
y = df['jamb_score']

# Разделение данных на train/validation/test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=1)

print(f"\nРазмеры выборок:")
print(f"Train: {X_train.shape}, {y_train.shape}")
print(f"Validation: {X_val.shape}, {y_val.shape}")
print(f"Test: {X_test.shape}, {y_test.shape}")

# Преобразование с помощью DictVectorizer
dv = DictVectorizer(sparse=True)

# Преобразуем тренировочные данные и применяем преобразование к валидационным и тестовым
X_train_dict = X_train.to_dict(orient='records')
X_val_dict = X_val.to_dict(orient='records')
X_test_dict = X_test.to_dict(orient='records')

X_train_encoded = dv.fit_transform(X_train_dict)
X_val_encoded = dv.transform(X_val_dict)
X_test_encoded = dv.transform(X_test_dict)

print(f"\nРазмеры после кодирования:")
print(f"Train: {X_train_encoded.shape}")
print(f"Validation: {X_val_encoded.shape}")
print(f"Test: {X_test_encoded.shape}")

## Вопрос 1: Дерево решений с max_depth=1
print("\n" + "="*50)
print("ВОПРОС 1: Дерево решений с max_depth=1")
print("="*50)

dt = DecisionTreeRegressor(max_depth=1, random_state=1)
dt.fit(X_train_encoded, y_train)

# Получаем имя признака, используемого для разбиения
feature_names = dv.get_feature_names_out()
tree_feature = feature_names[dt.tree_.feature[0]]

print(f"Признак, используемый для разбиения: {tree_feature}")

## Вопрос 2: Случайный лес с n_estimators=10
print("\n" + "="*50)
print("ВОПРОС 2: Случайный лес с n_estimators=10")
print("="*50)

rf = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)
rf.fit(X_train_encoded, y_train)

y_pred_val = rf.predict(X_val_encoded)
rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))

print(f"RMSE на валидационных данных: {rmse:.2f}")

## Вопрос 3: Подбор n_estimators (10-200)
print("\n" + "="*50)
print("ВОПРОС 3: Подбор n_estimators (10-200)")
print("="*50)

n_estimators_list = range(10, 201, 10)
best_rmse = float('inf')
best_n_estimators = 0
rmse_values = []

for n in n_estimators_list:
    rf = RandomForestRegressor(n_estimators=n, random_state=1, n_jobs=-1)
    rf.fit(X_train_encoded, y_train)

    y_pred_val = rf.predict(X_val_encoded)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
    rmse_values.append(rmse)

    if rmse < best_rmse:
        best_rmse = rmse
        best_n_estimators = n

    print(f"n_estimators={n}: RMSE = {rmse:.3f}")

# Находим точку, после которой улучшение незначительно
improvement_threshold = 0.001
stopping_point = n_estimators_list[0]

for i in range(1, len(rmse_values)):
    improvement = rmse_values[i-1] - rmse_values[i]
    if improvement < improvement_threshold:
        stopping_point = n_estimators_list[i]
        break

print(f"\nRMSE перестает значительно улучшаться после n_estimators = {stopping_point}")

## Вопрос 4: Подбор max_depth
print("\n" + "="*50)
print("ВОПРОС 4: Подбор max_depth")
print("="*50)

max_depth_values = [10, 15, 20, 25]
best_depth = None
best_avg_rmse = float('inf')

for depth in max_depth_values:
    rmse_scores = []

    for n in range(10, 201, 10):
        rf = RandomForestRegressor(
            n_estimators=n,
            max_depth=depth,
            random_state=1,
            n_jobs=-1
        )
        rf.fit(X_train_encoded, y_train)

        y_pred_val = rf.predict(X_val_encoded)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
        rmse_scores.append(rmse)

    avg_rmse = np.mean(rmse_scores)
    print(f"max_depth={depth}: Средний RMSE = {avg_rmse:.3f}")

    if avg_rmse < best_avg_rmse:
        best_avg_rmse = avg_rmse
        best_depth = depth

print(f"\nЛучшее значение max_depth: {best_depth}")

## Вопрос 5: Важность признаков
print("\n" + "="*50)
print("ВОПРОС 5: Важность признаков")
print("="*50)

rf_final = RandomForestRegressor(
    n_estimators=10,
    max_depth=20,
    random_state=1,
    n_jobs=-1
)
rf_final.fit(X_train_encoded, y_train)

# Получаем важность признаков
feature_importances = rf_final.feature_importances_
feature_names = dv.get_feature_names_out()

# Создаем DataFrame для удобного просмотра
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importances
}).sort_values('importance', ascending=False)

print("Топ-10 самых важных признаков:")
print(importance_df.head(10))

# Ищем интересующие нас признаки
target_features = ['study_hours_per_week', 'attendance_rate', 'distance_to_school', 'teacher_quality']
print(f"\nВажность интересующих нас признаков:")
for feature in target_features:
    # Ищем признак в закодированных названиях
    matching_features = [f for f in importance_df['feature'] if feature in f]
    if matching_features:
        for match in matching_features[:1]:  # Берем первый совпадающий
            importance = importance_df[importance_df['feature'] == match]['importance'].values[0]
            print(f"{match}: {importance:.4f}")

# Определяем самый важный из целевых признаков
most_important = None
max_importance = -1

for feature in target_features:
    matching_features = [f for f in importance_df['feature'] if feature in f]
    if matching_features:
        importance = importance_df[importance_df['feature'] == matching_features[0]]['importance'].values[0]
        if importance > max_importance:
            max_importance = importance
            most_important = feature

print(f"\nСамый важный признак из целевых: {most_important}")

Размер датасета: (5000, 17)

Первые 5 строк:
   jamb_score  study_hours_per_week  attendance_rate  teacher_quality  \
0         192                    22               78                4   
1         207                    14               88                4   
2         182                    29               87                2   
3         210                    29               99                2   
4         199                    12               98                3   

   distance_to_school school_type school_location extra_tutorials  \
0                12.4      Public           Urban             Yes   
1                 2.7      Public           Rural              No   
2                 9.6      Public           Rural             Yes   
3                 2.6      Public           Urban              No   
4                 8.8      Public           Urban              No   

  access_to_learning_materials parent_involvement it_knowledge  student_id  \
0                      

###**ФИНАЛЬНЫЕ ОТВЕТЫ НА ВОПРОСЫ**
Вопрос 1: study_hours_per_week, Вопрос 2: 42.13, Вопрос 3: 200, Вопрос 4: 10, Вопрос 5: study_hours_per_week.